In [ ]:
처음에 셀레니움으로 스크롤 다 내려서 top 100 보이게 한 후
top 100의 url 을 뷰티숩으로 하나씩 도출
각 url에 들어가서 (셀레니움으로 들어가서) 그 안에 내용들을 긁는다.
리뷰를 셀레니움으로 스크롤 다 내림
뷰티풀숩으로 내용 긁기

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys
import time
import click
from selenium.webdriver.common.action_chains import ActionChains
import sqlite3

#### 스크롤 마지막까지 내리기

In [2]:
from selenium.webdriver.common.keys import Keys

def scroll():
    body = driver.find_element_by_css_selector('body')
    for i in range(30):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)

#### 크롬드라이버 유니섹스향수 url 접속

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup as bsp

driver = webdriver.Chrome('C:/asi/project/chromedriver_win32 (3)/chromedriver')
url = ('https://www.glowpick.com/beauty/ranking?id=250&level=3')
driver.get(url)
driver.implicitly_wait(3)
scroll()
html = driver.page_source
src = bsp(html,'html.parser')

#### url 100개(top 100) 도출

In [13]:
perfumeranks = src.select('#gp-list > div > section.section-list > ul > li')
def top100():
    list_conts=[]
    for perfumerank in perfumeranks:
        urls = perfumerank.select('meta')[1]
        conts = urls['content']
        list_conts.append(conts)
    return list_conts

In [14]:
roll_top100s=top100()
print(roll_top100s)
#url 100개 다 집어넣음

['https://www.glowpick.com/product/24816', 'https://www.glowpick.com/product/10002', 'https://www.glowpick.com/product/15568', 'https://www.glowpick.com/product/122487', 'https://www.glowpick.com/product/11106', 'https://www.glowpick.com/product/93089', 'https://www.glowpick.com/product/80956', 'https://www.glowpick.com/product/96641', 'https://www.glowpick.com/product/84440', 'https://www.glowpick.com/product/109939', 'https://www.glowpick.com/product/1217', 'https://www.glowpick.com/product/15569', 'https://www.glowpick.com/product/77921', 'https://www.glowpick.com/product/8899', 'https://www.glowpick.com/product/80959', 'https://www.glowpick.com/product/106711', 'https://www.glowpick.com/product/109942', 'https://www.glowpick.com/product/58819', 'https://www.glowpick.com/product/90250', 'https://www.glowpick.com/product/109941', 'https://www.glowpick.com/product/113923', 'https://www.glowpick.com/product/117497', 'https://www.glowpick.com/product/16285', 'https://www.glowpick.com/pr

In [16]:
def scroll2():
    body = driver.find_element_by_css_selector('body')
    img = driver.find_element_by_css_selector('div.image__background-image')
    img.click()
    for i in range(100):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)

#### top 100 url 접속

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")


In [17]:
driver = webdriver.Chrome('C:/asi/project/chromedriver_win32 (3)/chromedriver')
kkk = 0
for roll_top100 in roll_top100s :
    driver.get(roll_top100)
    time.sleep(3)  # implicity 창이 안떠도 20초 기다리겠다.   ///  time sleep 화면 정지(시간만큼)
    scroll2()
    #click()
    whtml = driver.page_source
    wsrc = bsp(whtml,'html.parser')
    mains = wsrc.find(attrs={"class":'contents__product-info'})
    brand_name = mains.select('p.product-main-info__brand_name')[0].text.strip()
    #print(brand_name)
    name = mains.select('span.product-main-info__product_name__text')[0].text.strip()
    #print(name)
    note_= mains.select('div.info__description')[0].text.strip()
    note = re.sub('\*.*\.','',note_)
    #print(note)
    #print('*'*100)
    reviews = wsrc.select('li.review-list-item')
    for review in reviews :
        user = review.select('span.user-name')[0].text.strip()
        age_= review.select('span.txt')[0].text.strip()
        age = re.sub(' ·.*','',age_)
        gender = review.select('span.icon-sprite'+'{0}'.format('icon-gender-f'))
        if gender == review.select('span.icon-sprite'+'{0}'.format('icon-gender-f')):
            gender = '여'
        elif gender == review.select('span.icon-sprite'+'{0}'.format('icon-gender-m')):
            gender = '남'
        else:
            pass
        grade = review.select('span.icon-sprite'+'{0}'+'{1}'.format('rating-grade-icon','gpa-best-small'))   
        if grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-best-small')):
            grade = '5'
        elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-good-small')): 
            grade = '4'
        elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-soso-small')): 
            grade = '3'
        elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-bad-small')): 
            grade = '2'
        elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-worst-small')) :
            grade = '1'
        else:
            pass
        coment = review.select('p.review')[0].text.strip()
        #print(user,age,gender,grade,coment)
        #print('-'*100)
        d = (brand_name,name,note,user,age,gender,grade,coment)
        print(kkk)
        getData(d)
        print(d)
        kkk +=1
    time.sleep(3)

0
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '메꼬', '22세', '여', '5', '가격도 괜찮고 향도 좋아요!!')
1
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', "son's", '18세', '여', '5', '스트로베리아이스크림을 사용했는데,부드러운 딸기아이스크림향이 아니라싸구려 딸기 사탕향이에요. 엄청 진하고 달아요. 향을 잘못선택해서 별로지만제품만 놓고보면 좋아요. 롤온타입이라 간편하고, 오일이라 발향잘되고 소량으로 강한향나고 오래지속되고다른향으로 구입할마음있어요.')
2
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '누리네엄마', '27세', '여', '5', '피치는 달콤!설탕!막대사탕!츄잉껌! 이런 느낌이고 퍼지는 달달한 복숭아향~ 둘다 좋지만 좀 더 은은한 느낌의 퍼지가 더 좋다. 넘 좋아요b 뿌리는건 알콜향이 심하지만 이건 바르자마자 향기만 나니까 짱! 지속력은.. 그냥 자주 바르는걸로ㅋ')
3
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농

.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '매초롬', '30세', '여', '5', '휴대성 좋고 향도 넘나 좋지만 지속력..ㅂㄷㅂㄷ')
41
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '만두볼살', '19세', '여', '5', '오일에다가 용량도 괜찮은데 여름이 다가오는 날씨에는 향수변질 조심해야 하셔야할듯')
42
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '초코초코우유', '27세', '여', '5', '바르기 쉽고 가지고 다니기도 좋아요. 파우치에 넣어다니면서 생각날 때마다 바르면 됨. 지속력도 괜찮은편')
43
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '크료노스', '24세', '여', '5', '향이 은은해서 괜찮긴하지만 이게 종류마다 케바케라고 봄.바를 

.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '혀니짱', '32세', '여', '5', '향수사는데 피치향 껴주길래 써봤는데 향도 좋고 귀여워용 ^^ 가볍게 손목에 돌돌 귀뒤에 돌돌 ^^')
88
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '쩰리', '30세', '여', '5', '파우치에 쏙 넣어서 쓰기 딱 좋아요사람들 많은곳에서도 그냥 내몸에 슥슥 문질러주면 되니까 쓰기도 편해요근데 역시나 지속력이 그렇게 좋지 않아서 아쉽..')
89
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 다른 향들과 믹스매치해서 사용 가능', '아름다워', '24세', '여', '5', '베이비파우더 사용했는데 뚜껑이 약간 부실해서 찔끔찔끔나왔던게 아쉽다향은 내가생각했던것과 달랐지만 잔향이 너무 좋았고 첫향은 약간 독했다')
90
.('데메테르 (DEMETER)', '롤온 퍼퓸오일', '- 무알코올로 강한 지속력의 포뮬레이션을 가진 데메테르의 퍼퓸 오일은 고농축 오일로 7시간 이상 지속- 사용하기 쉽도록 제작된 롤온 스타일! 잇꽃오일, 호호바오일, 비타민 E가 함유되어 당신의 피부를 부드럽게 지켜줍니다.- 

127
.('프레쉬 (fresh)', '슈가 리치 오드퍼퓸 롤온', "세계적인 조향사이자 프레쉬의 창시자인 레브 글레이즈먼(Lev Glazman)에 의해 탄생된 '액체의 보석'이라 불리우는 순수한 향수(달콤하고 낭만적인 프루티 향) / 휴대하기 쉬운 롤러볼 타입이국적인 과일 향을 주 원료로 캬라멜과 연꽃의 플로럴 향과 상큼한 시트러스의 향을 배합한 부드러운 향입니다.탑 노트 : 자몽, 이탈리안 레몬, 라임 블라썸하트 노트 : 리치, 연꽃, 프리지아베이스 노트 : 샌달우드, 통카빈, 엠버", '민감피부공돌', '31세', '여', '5', '봄에 쓰기 너무 좋은향 휴대도 간편해서 좋아요')
128
.('프레쉬 (fresh)', '슈가 리치 오드퍼퓸 롤온', "세계적인 조향사이자 프레쉬의 창시자인 레브 글레이즈먼(Lev Glazman)에 의해 탄생된 '액체의 보석'이라 불리우는 순수한 향수(달콤하고 낭만적인 프루티 향) / 휴대하기 쉬운 롤러볼 타입이국적인 과일 향을 주 원료로 캬라멜과 연꽃의 플로럴 향과 상큼한 시트러스의 향을 배합한 부드러운 향입니다.탑 노트 : 자몽, 이탈리안 레몬, 라임 블라썸하트 노트 : 리치, 연꽃, 프리지아베이스 노트 : 샌달우드, 통카빈, 엠버", '프시', '18세', '여', '5', '달달하고 포근한 향이에요 향이 좀 세서 겨울에 좋을 것 같늠 롤온타입이어도 지속력이나 향 나는 거 괜찮고 종이 케이스다 예뻐요')
129
.('프레쉬 (fresh)', '슈가 리치 오드퍼퓸 롤온', "세계적인 조향사이자 프레쉬의 창시자인 레브 글레이즈먼(Lev Glazman)에 의해 탄생된 '액체의 보석'이라 불리우는 순수한 향수(달콤하고 낭만적인 프루티 향) / 휴대하기 쉬운 롤러볼 타입이국적인 과일 향을 주 원료로 캬라멜과 연꽃의 플로럴 향과 상큼한 시트러스의 향을 배합한 부드러운 향입니다.탑 노트 : 자몽, 이탈리안 레몬, 라임 블라썸하트 노트 : 리치, 연꽃, 프리지아베이스 노트 : 샌달우드, 통카빈, 엠버", '숭숭아', '30세', '여

154
.('디올 (DIOR)', '미스 디올 블루밍 부케 롤러-펄 오 드 뚜왈렛', '마치 산뜻한 플로럴 향에 휩싸인 듯 우아하고 아름다운 미스 디올 오 드 뚜왈렛의 향기그라스산 로즈왕 은방울꽃이 사랑의 왈츠를 추듯 감미롭게 어우러진 미스 디올 롤러-펄 오드 뚜왈렛은 삶과 사랑에 빠진 미스 디올의 사랑스러움을 표현합니다.', '루피는화가나', '26세', '여', '5', '향은 말해뭐해 진짜 짱. 향수도 있지만 들고나가기 부담스러우니까 이게 딱! 좋은데 이것도 살짝 무거운감은 있어요. 미니백 좋아하시면 쫌 클듯해요, 하지만 그래도 향이 너무 좋으니 가지고 다닐만해요 굳굳 병도 예쁘고!')
155
.('디올 (DIOR)', '미스 디올 블루밍 부케 롤러-펄 오 드 뚜왈렛', '마치 산뜻한 플로럴 향에 휩싸인 듯 우아하고 아름다운 미스 디올 오 드 뚜왈렛의 향기그라스산 로즈왕 은방울꽃이 사랑의 왈츠를 추듯 감미롭게 어우러진 미스 디올 롤러-펄 오드 뚜왈렛은 삶과 사랑에 빠진 미스 디올의 사랑스러움을 표현합니다.', 'Nimos', '27세', '여', '5', '백화점 1층냄새에요 고급스럽고 페미닌한 향이에요 강한 향수를 싫어하시면 싫어할 수도 있는데 잔향이 아주 매혹적이라 좋아요 쓰다가 어머니께 뺏겼어요 ㅋㅋㅋㅋ 향이 좋다고 노래를 부르셔서ㅠㅠ')
156
.('디올 (DIOR)', '미스 디올 블루밍 부케 롤러-펄 오 드 뚜왈렛', '마치 산뜻한 플로럴 향에 휩싸인 듯 우아하고 아름다운 미스 디올 오 드 뚜왈렛의 향기그라스산 로즈왕 은방울꽃이 사랑의 왈츠를 추듯 감미롭게 어우러진 미스 디올 롤러-펄 오드 뚜왈렛은 삶과 사랑에 빠진 미스 디올의 사랑스러움을 표현합니다.', '뷰스타랴옹', '38세', '여', '5', '롤러로 되어있어서 휴대용으로 사용하기에 좋아요.블루밍 부케 향이 유명한만큼 향이 샤랄라 꽃향기라 좋네요.바로 샤워하고 나왔을 때 나는 꽃향기처럼상쾌하고 기분이 좋아지는 향기네요.디올 향수중에서 가장 좋은 향인것 같아요.향이 진하지 않고 머리 안

188
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (바이올렛)', '-탑 : 벨플라워, 클로버-미들 : 은방울꽃, 로즈힙, 자스민-베이스 : 머스크, 화이트 앰버', '먕미', '21세', '여', '5', '이 향은 여성스럽고 또 엄청 무겁지도 않으면서 암튼 깨끗하고 예쁜 여학생 한테 날 것 같은 향이에요! 향수를 잘 몰라서 탑 미들 베이스 이런건 모르겠고 아마 공홈에 가면 나와있겠져? 저는 이거 거의 다 썼습니다 좋아서 ㅎㅎ 근데 약간 샜던거같아요 용기만 개선해준다면 참 좋겠네요 근데 제가 산지 좀 돼서 이미 개선이 됐을 수도 있겠어요 무려 2016년쯤..? 산거같은데.. 암튼 향은 좋습니다 ㅎㅎ 향은 안바뀌였겠져 머')
189
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (바이올렛)', '-탑 : 벨플라워, 클로버-미들 : 은방울꽃, 로즈힙, 자스민-베이스 : 머스크, 화이트 앰버', 'lads', '31세', '여', '5', '시원하고 향긋한 향이 난다. 살짝 달달해서 좋음')
190
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (바이올렛)', '-탑 : 벨플라워, 클로버-미들 : 은방울꽃, 로즈힙, 자스민-베이스 : 머스크, 화이트 앰버', '쀼빗', '23세', '여', '5', '왜 이름이 바이올렛인지 모르겠지만 이거만 바르면 뜨거운 여름에 수박화채가 떠오름..지속력은 오래가는 편')
191
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (바이올렛)', '-탑 : 벨플라워, 클로버-미들 : 은방울꽃, 로즈힙, 자스민-베이스 : 머스크, 화이트 앰버', '영양제', '20세', '여', '5', '뿌리는형식이아니라  그 문질문질해야하는건데  바르면 일단 향이 찐해서 별로안좋은데 냄새는 오래가서 좋음 다른향수 향은 그닥 안좋은데 애가 좋음  요거 향은  누가맡아도 퍼퓸이라고 할  정도록 향   이름과 똑같음')
192
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (바이올렛)', '-탑 : 벨플라워, 클로버-미들 :

214
.('프레쉬 (fresh)', '씨트론 드 빈 오드퍼퓸 롤온', "세계적인 조향사이자 프레쉬의 창시자인 레브 글레이즈먼(Lev Glazman)에 의해 탄생된 '액체의 보석'이라 불리우는 자연 그대로의 향을 재현한 향수톡 쏘는 샴페인과 같은 스파클링하면서 따뜻한 느낌의 남녀 공용 향/ 휴대하기 쉬운 롤러볼 타입씨트론 드 빈 오드퍼퓸은 프랑스 랭스 지방의 아름다운 단상에서 영감을 받아 만들어진 향수입니다.샴페인처럼 생동감있고 신선하지만 가볍지 않은, 럭셔리하고 깊은 흙내음이 함께 어우려져 남녀가 함께 사용하기 좋습니다.첫 향은 샴페인과 같이 스파클링하고, 그윽한 하트 노트로 이어지며 마침내 우디 계열의 따뜻한 잔향이 남게됩니다.[노트]탑 노트: 톡톡튀는 시트러스가 샴페인의 기포와 같은 느낌을 주는 첫향하트 노트: 피노누와, 자스민의 그윽한 하트 노트베이스 노트: 우디계열이 주는 포도정원의 따뜻한 흙내음새", '메렁이닷', '29세', '여', '5', '향이 너무 조음과하지 않고 머리아프지 않음근데 잘 묻히고 금방 사라지는 편')
215
.('프레쉬 (fresh)', '씨트론 드 빈 오드퍼퓸 롤온', "세계적인 조향사이자 프레쉬의 창시자인 레브 글레이즈먼(Lev Glazman)에 의해 탄생된 '액체의 보석'이라 불리우는 자연 그대로의 향을 재현한 향수톡 쏘는 샴페인과 같은 스파클링하면서 따뜻한 느낌의 남녀 공용 향/ 휴대하기 쉬운 롤러볼 타입씨트론 드 빈 오드퍼퓸은 프랑스 랭스 지방의 아름다운 단상에서 영감을 받아 만들어진 향수입니다.샴페인처럼 생동감있고 신선하지만 가볍지 않은, 럭셔리하고 깊은 흙내음이 함께 어우려져 남녀가 함께 사용하기 좋습니다.첫 향은 샴페인과 같이 스파클링하고, 그윽한 하트 노트로 이어지며 마침내 우디 계열의 따뜻한 잔향이 남게됩니다.[노트]탑 노트: 톡톡튀는 시트러스가 샴페인의 기포와 같은 느낌을 주는 첫향하트 노트: 피노누와, 자스민의 그윽한 하트 노트베이스 노트: 우디계열이 주는 포도정원의 따뜻한 흙내음새", '겨울잠', '39

244
.('헤라 (HERA)', '터치 오브 헤라 오렌지 립스틱 오 드 뚜왈렛 롤러볼', '언제 어디서나 쉽게 나만의 향기를 완성하는 롤러볼 향수- 탑 노트: 당장 터져 나올 것만 같은 오렌지와 초록 자몽의 싱싱함 (플로리다 오렌지, 그린 자몽, 스피아민트, 이집트 바질)- 미들 노트: 상쾌함 속 숨겨져 있던! 예상치 못한 플로럴 노트의 메들리 (터키 로즈, 매그놀리아, 바이올렛)- 베이스 노트: 폭발할 것 같았던 처음을 따뜻하게 감싸주는 편안한 베이스 (앰버, 머스크)', '1119', '29세', '여', '5', '진하지도않고 무겁지도 않아서 쉽게 쓱쓱 잘 사용하고 있어요. 밖에서도 바르기 편하고 은근히 오래 쓸수있어서 산지 몇달 됐는데 아직도 쓰고있어요.지속력은 좋은 편은 아니지만 향수가 어려운 저한테는 너무 진하게 오래가는거보다는 이렇게 가볍게 쓸수있는게 더 좋은것같아요')
245
.('헤라 (HERA)', '터치 오브 헤라 오렌지 립스틱 오 드 뚜왈렛 롤러볼', '언제 어디서나 쉽게 나만의 향기를 완성하는 롤러볼 향수- 탑 노트: 당장 터져 나올 것만 같은 오렌지와 초록 자몽의 싱싱함 (플로리다 오렌지, 그린 자몽, 스피아민트, 이집트 바질)- 미들 노트: 상쾌함 속 숨겨져 있던! 예상치 못한 플로럴 노트의 메들리 (터키 로즈, 매그놀리아, 바이올렛)- 베이스 노트: 폭발할 것 같았던 처음을 따뜻하게 감싸주는 편안한 베이스 (앰버, 머스크)', '유지니지니', '22세', '여', '5', '선물 받아서 사용중인데 은은하고 파우치에 들고다녀도 가볍고 해서 너무 좋아요')
246
.('헤라 (HERA)', '터치 오브 헤라 오렌지 립스틱 오 드 뚜왈렛 롤러볼', '언제 어디서나 쉽게 나만의 향기를 완성하는 롤러볼 향수- 탑 노트: 당장 터져 나올 것만 같은 오렌지와 초록 자몽의 싱싱함 (플로리다 오렌지, 그린 자몽, 스피아민트, 이집트 바질)- 미들 노트: 상쾌함 속 숨겨져 있던! 예상치 못한 플로럴 노트의 메들리 (터키 로즈, 매그놀리아, 바이올렛

268
.('쥴리크 (jurlique)', '에센스 오브 로즈 롤 온 프레그란스', '쥴리크가 직접 재배한 로즈로 만든 우아하고 로맨틱한 향의 여성용 향수쥴리크가 직접 재배한 오가닉 로즈의 우아하고 로맨틱한 향이 오랫동안 지속되는 여성용 향수입니다. 쥴리크 No.1 시그니쳐 로즈의 향수로 모든 피부 타입에 사용가능합니다. 휴대용 사이즈와 사용하기 간편한 롤링 제품으로 언제 어디서나 매혹적인 로즈향을 더할수 있습니다.', 'aaaaaaw2', '31세', '여', '5', '호주 여행 다녀오는 길에 대학생 되는 동생 선물로 사다준 제품은은하지만 확실한 장미향이고 어린 나이에 쓰기에도 부담스럽지 않을 향이라 좋았어요롤온 타입이라 휴대와 어플라이가 간편하고 양조절도 쉬워서 향수 처음 쓰는 동생에게 딱이라 생각했음롤온 타입 제품들은 가끔 쓰다보면 새어나와 파우치를 망칠 때가 있던데 그런 일도 없었다고 하고 데일리로 항상 쓴다며 후기가 좋았네요 ㅋㅋㅋ')
269
.('데메테르 (DEMETER)', '크리스탈 로즈 오드뚜왈렛 롤온 퍼퓸', '소프트 로지 플로럴 상쾌하면서도 관능적인 프루티향이 느껴지는 모던하고 여성스러운 향의 조합- 탑 노트: 애플, 베르가모트, 아쿠아틱- 미들 노트: 로즈, 자스민, 네롤리- 베이스 노트: 머스크, 앰버, 우디', '벚꽃축제', '21세', '여', '5', '-진짜 향 하나로 모든 단점을 커버할 수 있음-향수만 뿌리고 나오면 지속력이 짧지만 외출 중에 이거 수시로 바르면 됨-디자인도 너무 예쁘고 일단 향이 진짜 최고임 진짜 싫다는 사람 한번도 못봄-개인적으로는 향 오래가는 바디미스트 버전으로도 나왔으면')
270
.('데메테르 (DEMETER)', '스윗베이비 오드뚜왈렛 롤온 퍼퓸', '여성들이 사랑하는 프루티 플로럴 베스트 향기- 탑 노트: 그린 애플, 탠저린- 미들 노트: 라일락, 자스민, 로즈, 블랙 커런트- 베이스 노트: 우디, 앰버, 머스크', '아르미썰!', '31세', '여', '5', '테스터로 슥 맡아보고 바로 반해서 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.image__background-image"}
  (Session info: chrome=89.0.4389.90)


In [6]:
def getData(d): # 데이터베이스에 데이터 집어넣기
    dbname='glowpick.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='INSERT INTO glow (brand_name,name,note,user,age,gender,grade,coment) VALUES(?,?,?,?,?,?,?,? )' 
        res=cursor.execute(sql,d)
        print('.',end='')

In [ ]:
############################################################

In [19]:
driver = webdriver.Chrome('C:/asi/project/chromedriver_win32 (3)/chromedriver')
url = ('https://www.glowpick.com/product/80957')
driver.get(url)
driver.implicitly_wait(3)
scroll()
html = driver.page_source
src = bsp(html,'html.parser')
time.sleep(3)  # implicity 창이 안떠도 20초 기다리겠다.   ///  time sleep 화면 정지(시간만큼)
scroll2()
#click()
whtml = driver.page_source
wsrc = bsp(whtml,'html.parser')
mains = wsrc.find(attrs={"class":'contents__product-info'})
brand_name = mains.select('p.product-main-info__brand_name')[0].text.strip()
#print(brand_name)
name = mains.select('span.product-main-info__product_name__text')[0].text.strip()
#print(name)
note_= mains.select('div.info__description')[0].text.strip()
note = re.sub('\*.*\.','',note_)
#print(note)
#print('*'*100)
reviews = wsrc.select('li.review-list-item')
for review in reviews :
    user = review.select('span.user-name')[0].text.strip()
    age_= review.select('span.txt')[0].text.strip()
    age = re.sub(' ·.*','',age_)
    gender = review.select('span.icon-sprite'+'{0}'.format('icon-gender-f'))
    if gender == review.select('span.icon-sprite'+'{0}'.format('icon-gender-f')):
        gender = '여'
    elif gender == review.select('span.icon-sprite'+'{0}'.format('icon-gender-m')):
        gender = '남'
    else:
        pass
    grade = review.select('span.icon-sprite'+'{0}'+'{1}'.format('rating-grade-icon','gpa-best-small'))   
    if grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-best-small')):
        grade = '5'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-good-small')): 
        grade = '4'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-soso-small')): 
        grade = '3'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-bad-small')): 
        grade = '2'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-worst-small')) :
        grade = '1'
    else:
        pass
    coment = review.select('p.review')[0].text.strip()
    #print(user,age,gender,grade,coment)
    #print('-'*100)
    d = (brand_name,name,note,user,age,gender,grade,coment)
    print(kkk)
    getData(d)
    print(d)
    kkk +=1
time.sleep(3)

277
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (자스민)', '-탑 : 만다린, 네롤리, 베르가못-미들 : 자스민, 라벤더, 바이올렛-베이스 : 자스민 티, 머스크', '서토리._.', '20세', '여', '5', '자스민 향기는 아니고 꽃 향기와 풀 냄새가 나요꽃 향기를 좋아해 마음에 들었지만 자스민 향기를 생각하고 주문하시면 안 될 것 같아요 지속력이 짧아 아쉬운 제품입니다 디자인은 예뻐요')
278
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (자스민)', '-탑 : 만다린, 네롤리, 베르가못-미들 : 자스민, 라벤더, 바이올렛-베이스 : 자스민 티, 머스크', 'lads', '31세', '여', '5', '다들 공통적으로 하는 말처럼.. 자스민은 아님. 근데 좋은 풀향')
279
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (자스민)', '-탑 : 만다린, 네롤리, 베르가못-미들 : 자스민, 라벤더, 바이올렛-베이스 : 자스민 티, 머스크', 'stephyoon', '23세', '여', '5', '어퓨 롤온향수 하나하나 다 맡아보고 고른게 이거에요후각이 예민해서 좀 까다로운데 유일하게 통과한게 자스민입니당😮 (바이올렛도 청순하고 연보라색이 떠오르는 느낌이라 예쁜향이긴한데 흔한향이었어요 바질은 애매했구요)솔직히 자스민향 거의 안느껴지고 여러가지 향기가 조화롭게 섞인향이에요코코넛이 살짝 무겁고 부드럽게 깔려있고 그 위로 라임이랑 하얀꽃들이 느껴지네요!저렴한 가격에 비해 롤온용기의 품질이 괜찮고 휴대하면서 평소에 쓰는 향수에 레이어드하기도 좋은것같아요')
280
.('어퓨 (Apieu)', '마이 핸디 롤온 퍼퓸 (자스민)', '-탑 : 만다린, 네롤리, 베르가못-미들 : 자스민, 라벤더, 바이올렛-베이스 : 자스민 티, 머스크', '정명훈이이상형', '18세', '여', '5', '자스민이라는 말에 혹해 샀는데 자스민 향은 아니에요. 확실히 시원하고 약간 풀향?스러운 향이 나요. 다행이 막 어후 쉣 하는 정도는 아니고 은은한 풀

In [32]:
driver = webdriver.Chrome('C:/asi/project/chromedriver_win32 (3)/chromedriver')
url = ('https://www.glowpick.com/product/110699')
driver.get(url)
driver.implicitly_wait(3)
scroll()
html = driver.page_source
src = bsp(html,'html.parser')
time.sleep(3)  # implicity 창이 안떠도 20초 기다리겠다.   ///  time sleep 화면 정지(시간만큼)
scroll2()
#click()
whtml = driver.page_source
wsrc = bsp(whtml,'html.parser')
mains = wsrc.find(attrs={"class":'contents__product-info'})
brand_name = mains.select('p.product-main-info__brand_name')[0].text.strip()
#print(brand_name)
name = mains.select('span.product-main-info__product_name__text')[0].text.strip()
#print(name)
note_= mains.select('div.info__description')[0].text.strip()
note = re.sub('\*.*\.','',note_)
#print(note)
#print('*'*100)
reviews = wsrc.select('li.review-list-item')
for review in reviews :
    user = review.select('span.user-name')[0].text.strip()
    age_= review.select('span.txt')[0].text.strip()
    age = re.sub(' ·.*','',age_)
    gender = review.select('span.icon-sprite'+'{0}'.format('icon-gender-f'))
    if gender == review.select('span.icon-sprite'+'{0}'.format('icon-gender-f')):
        gender = '여'
    elif gender == review.select('span.icon-sprite'+'{0}'.format('icon-gender-m')):
        gender = '남'
    else:
        pass
    grade = review.select('span.icon-sprite'+'{0}'+'{1}'.format('rating-grade-icon','gpa-best-small'))   
    if grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-best-small')):
        grade = '5'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-good-small')): 
        grade = '4'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-soso-small')): 
        grade = '3'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-bad-small')): 
        grade = '2'
    elif grade == review.select('span.icon-sprite'+'{0}'+'{1}'.format('icon-gender-f','gpa-worst-small')) :
        grade = '1'
    else:
        pass
    coment = review.select('p.review')[0].text.strip()
    #print(user,age,gender,grade,coment)
    #print('-'*100)
    d = (brand_name,name,note,user,age,gender,grade,coment)
    print(kkk)
    getData(d)
    print(d)
    kkk +=1
time.sleep(3)

321
.('뱅네프생토노레 (29 ST HONORE)', '솔리드 에쌍스 파퓨메 로즈 앤 베이', '장미의 은은하고 달콤한 향과 블랙커런트, 베르가못, 만다린의 풍성한 푸케향을 담은 향수- 휴대하기 편한 고체스틱의 형태로 담은 제품- 자연 유래 성분으로 보습까지 충전해주는 멀티퍼퓸- 저자극 성분', '문곰문', '42세', '여', '5', '휴대하기 편하다는 장점이외엔향수로 장점이 없음로즈앤베이 구매한건데향은 쏘쏘하나 지속력 제로임아 자석식 용기하난 맘에듬')


In [24]:
def scroll3():
    body = driver.find_element_by_css_selector('body')
    img = driver.find_element_by_css_selector('div.image__background-image')
    img.click()
    for i in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)